# ALENN - Replication Notebook 
## Random Walk Model, MDN

Donovan Platt
<br>
Mathematical Institute, University of Oxford
<br>
Institute for New Economic Thinking at the Oxford Martin School
<br>
<br>
Copyright (c) 2020, University of Oxford. All rights reserved.
<br>
Distributed under a BSD 3-Clause licence. See the accompanying LICENCE file for further details.

# 1. Modules and Packages
Load all required modules and packages.

In [1]:
# Import the ALENN ABM Estimation Package
import alenn

# Import Numerical Computation Libraries
import numpy as np
import pandas as pd

# Import General Mathematical Libraries
from scipy import stats

# Import System Libraries
import os
import logging

Using TensorFlow backend.


In [2]:
# Disable Tensorflow Deprecation Warnings
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Tensorflow 2.x deprecates many Tensorflow 1.x methods, causing Tensorflow 1.15.0 to output a large number 
# of deprecation warnings when performing the first likelihood calculation. This can be very distracting,
# leading us to disable them.

# 2. Estimation Experiments
Replication of the MDN experiments. Note that here we generate only a single Markov Chain as opposed to the 5 considered in the original paper.

## 2.1. Free Parameter Set 1

### Model Specification

In [12]:
# Specify the Simulated Data Characteristics
T_emp = 1000    # Pseudo-empirical series length 
T_sim = 1000    # Length of each Monte Carlo replication
n = 100         # Number of Monte Carlo replications

# Specify the Pseudo-Empirical Data
empirical = np.diff(alenn.models.random_walk(700, 0.4, 0.5, 1, 2, T_emp, 1, 1), axis = 0)[:, 0]

# Define the Candidate Model Function
def model(theta):
    return np.diff(alenn.models.random_walk(700, 0.4, 0.5, theta[0], theta[1], T_sim, n, 7), axis = 0)

# Define Parameter Priors
priors = [stats.uniform(loc = 0, scale = 10).pdf,
          stats.uniform(loc = 0, scale = 10).pdf]

# Define the Parameter Bounds
theta_lower = np.array([0, 0])
theta_upper = np.array([10, 10])

### Posterior Specification

In [4]:
# Create an MDN Posterior Approximator Object (Uses Default Settings from the Paper)
posterior = alenn.mdn.MDNPosterior()

# Add the Model, Priors, and Empirical Data to the Newly-created Object
posterior.set_model(model)
posterior.set_prior(priors)
posterior.load_data(empirical)

--------------------------------------
Successfully created a new MDN object:
--------------------------------------
Number of lags:                3                   
Number of mixture components:  16                  
Number of neurons per layer:   32                  
Number of hidden layers:       3                   
Batch size:                    512                 
Number of epochs:              12                  
Activation function:           relu                
Input noise:                   0.2                 
Output noise:                  0.2                 
--------------------------------------

Model function successfully set.
----------------------------------------------------------------------------

Model prior successfully set. The model has 2 free parameters.
----------------------------------------------------------------------------

Empirical data successfully loaded. There are 999 observations in total.
--------------------------------------------------

### Sampler Specification

In [13]:
# Create an Adaptive MCMC Sampler Object
sampler = alenn.mcmc.AdaptiveMCMC(K = 70, S = 5000)

# Add the Posterior Approximator and Parameter Ranges to the Newly-created Object
sampler.set_posterior(posterior)
sampler.set_initialisation_ranges(theta_lower, theta_upper)

# Initiate the Sampling Process
sampler.sample_posterior()

-----------------------------------------------
Successfully created a new MCMC sampler object:
-----------------------------------------------
Number of sample sets:         5000                
Number of samples per set:     70                  
-----------------------------------------------

MDNPosterior object successfully loaded.
----------------------------------------------------------------------------

Initialisation ranges successfully set.

           Lower Bound  Upper Bound
Parameter                          
1                    0           10
2                    0           10
----------------------------------------------------------------------------



### Result Processing

In [6]:
# Process the Sampler Output
samples = sampler.process_samples(burn_in = 1500)

# Calculate the Posterior Mean
pos_mean = samples[:, :posterior.num_param].mean(axis = 0)

# Calculate the Posterior Standard Deviation
pos_std = samples[:, :posterior.num_param].std(axis = 0)

# Construct a Result Table
result_table = pd.DataFrame(np.array([pos_mean, pos_std]).transpose(), columns = ['Posterior Mean', 'Posterior Std. Dev.'])
result_table.index.name = 'Parameter'
result_table.index += 1

# Display the Result Table
print('Final Estimation Results:')
print('')
print(result_table)

Final Estimation Results:

           Posterior Mean  Posterior Std. Dev.
Parameter                                     
1                0.946336             0.034519
2                1.950764             0.097562


## 2.2. Free Parameter Set 2

### Model Specification

In [14]:
# Specify the Simulated Data Characteristics
T_emp = 1000    # Pseudo-empirical series length 
T_sim = 1000    # Length of each Monte Carlo replication
n = 100         # Number of Monte Carlo replications

# Specify the Pseudo-Empirical Data
empirical = np.diff(alenn.models.random_walk(700, 0.1, 0.2, 1, 2, T_emp, 1, 1), axis = 0)[:, 0]

# Define the Candidate Model Function
def model(theta):
    return np.diff(alenn.models.random_walk(700, 0.1, 0.2, theta[0], theta[1], T_sim, n, 7), axis = 0)

# Define Parameter Priors
priors = [stats.uniform(loc = 0, scale = 10).pdf,
          stats.uniform(loc = 0, scale = 10).pdf]

# Define the Parameter Bounds
theta_lower = np.array([0, 0])
theta_upper = np.array([10, 10])

### Posterior Specification

In [4]:
# Create an MDN Posterior Approximator Object (Uses Default Settings from the Paper)
posterior = alenn.mdn.MDNPosterior()

# Add the Model, Priors, and Empirical Data to the Newly-created Object
posterior.set_model(model)
posterior.set_prior(priors)
posterior.load_data(empirical)

--------------------------------------
Successfully created a new MDN object:
--------------------------------------
Number of lags:                3                   
Number of mixture components:  16                  
Number of neurons per layer:   32                  
Number of hidden layers:       3                   
Batch size:                    512                 
Number of epochs:              12                  
Activation function:           relu                
Input noise:                   0.2                 
Output noise:                  0.2                 
--------------------------------------

Model function successfully set.
----------------------------------------------------------------------------

Model prior successfully set. The model has 2 free parameters.
----------------------------------------------------------------------------

Empirical data successfully loaded. There are 999 observations in total.
--------------------------------------------------

### Sampler Specification

In [15]:
# Create an Adaptive MCMC Sampler Object
sampler = alenn.mcmc.AdaptiveMCMC(K = 70, S = 5000)

# Add the Posterior Approximator and Parameter Ranges to the Newly-created Object
sampler.set_posterior(posterior)
sampler.set_initialisation_ranges(theta_lower, theta_upper)

# Initiate the Sampling Process
sampler.sample_posterior()

-----------------------------------------------
Successfully created a new MCMC sampler object:
-----------------------------------------------
Number of sample sets:         5000                
Number of samples per set:     70                  
-----------------------------------------------

MDNPosterior object successfully loaded.
----------------------------------------------------------------------------

Initialisation ranges successfully set.

           Lower Bound  Upper Bound
Parameter                          
1                    0           10
2                    0           10
----------------------------------------------------------------------------



### Result Processing

In [6]:
# Process the Sampler Output
samples = sampler.process_samples(burn_in = 1500)

# Calculate the Posterior Mean
pos_mean = samples[:, :posterior.num_param].mean(axis = 0)

# Calculate the Posterior Standard Deviation
pos_std = samples[:, :posterior.num_param].std(axis = 0)

# Construct a Result Table
result_table = pd.DataFrame(np.array([pos_mean, pos_std]).transpose(), columns = ['Posterior Mean', 'Posterior Std. Dev.'])
result_table.index.name = 'Parameter'
result_table.index += 1

# Display the Result Table
print('Final Estimation Results:')
print('')
print(result_table)

Final Estimation Results:

           Posterior Mean  Posterior Std. Dev.
Parameter                                     
1                0.945404             0.039242
2                1.946920             0.109503


## 2.3. Free Parameter Set 3

### Model Specification

In [4]:
# Specify the Simulated Data Characteristics
T_emp = 1000    # Pseudo-empirical series length 
T_sim = 1000    # Length of each Monte Carlo replication
n = 100         # Number of Monte Carlo replications

# Specify the Pseudo-Empirical Data
empirical = np.diff(alenn.models.random_walk(700, 0.4, 0.5, 1, 2, T_emp, 1, 1), axis = 0)[:, 0]

# Define the Candidate Model Function
def model(theta):
    return np.diff(alenn.models.random_walk(700, theta[0], theta[1], 1, 2, T_sim, n, 7), axis = 0)

# Define Parameter Priors
priors = [stats.uniform(loc = -2, scale = 4).pdf,
          stats.uniform(loc = -2, scale = 4).pdf]

# Define the Parameter Bounds
theta_lower = np.array([-2, -2])
theta_upper = np.array([2, 2])

### Posterior Specification

In [5]:
# Create an MDN Posterior Approximator Object (Uses Default Settings from the Paper)
posterior = alenn.mdn.MDNPosterior()

# Add the Model, Priors, and Empirical Data to the Newly-created Object
posterior.set_model(model)
posterior.set_prior(priors)
posterior.load_data(empirical)

--------------------------------------
Successfully created a new MDN object:
--------------------------------------
Number of lags:                3                   
Number of mixture components:  16                  
Number of neurons per layer:   32                  
Number of hidden layers:       3                   
Batch size:                    512                 
Number of epochs:              12                  
Activation function:           relu                
Input noise:                   0.2                 
Output noise:                  0.2                 
--------------------------------------

Model function successfully set.
----------------------------------------------------------------------------

Model prior successfully set. The model has 2 free parameters.
----------------------------------------------------------------------------

Empirical data successfully loaded. There are 999 observations in total.
--------------------------------------------------

### Sampler Specification

In [6]:
# Create an Adaptive MCMC Sampler Object
sampler = alenn.mcmc.AdaptiveMCMC(K = 70, S = 5000)

# Add the Posterior Approximator and Parameter Ranges to the Newly-created Object
sampler.set_posterior(posterior)
sampler.set_initialisation_ranges(theta_lower, theta_upper)

# Initiate the Sampling Process
sampler.sample_posterior()

-----------------------------------------------
Successfully created a new MCMC sampler object:
-----------------------------------------------
Number of sample sets:         5000                
Number of samples per set:     70                  
-----------------------------------------------

MDNPosterior object successfully loaded.
----------------------------------------------------------------------------

Initialisation ranges successfully set.

           Lower Bound  Upper Bound
Parameter                          
1                   -2            2
2                   -2            2
----------------------------------------------------------------------------



### Result Processing

In [10]:
# Process the Sampler Output
samples = sampler.process_samples(burn_in = 1500)

# Calculate the Posterior Mean
pos_mean = samples[:, :posterior.num_param].mean(axis = 0)

# Calculate the Posterior Standard Deviation
pos_std = samples[:, :posterior.num_param].std(axis = 0)

# Construct a Result Table
result_table = pd.DataFrame(np.array([pos_mean, pos_std]).transpose(), columns = ['Posterior Mean', 'Posterior Std. Dev.'])
result_table.index.name = 'Parameter'
result_table.index += 1

# Display the Result Table
print('Final Estimation Results:')
print('')
print(result_table)

Final Estimation Results:

           Posterior Mean  Posterior Std. Dev.
Parameter                                     
1                0.486288             0.038618
2                0.544431             0.110380


## 2.4. Free Parameter Set 4

### Model Specification

In [3]:
# Specify the Simulated Data Characteristics
T_emp = 1000    # Pseudo-empirical series length 
T_sim = 1000    # Length of each Monte Carlo replication
n = 100         # Number of Monte Carlo replications

# Specify the Pseudo-Empirical Data
empirical = np.diff(alenn.models.random_walk(700, 0.4, 0.7, 1, 2, T_emp, 1, 1), axis = 0)[:, 0]

# Define the Candidate Model Function
def model(theta):
    return np.diff(alenn.models.random_walk(700, theta[0], theta[1], 1, 2, T_sim, n, 7), axis = 0)

# Define Parameter Priors
priors = [stats.uniform(loc = -2, scale = 4).pdf,
          stats.uniform(loc = -2, scale = 4).pdf]

# Define the Parameter Bounds
theta_lower = np.array([-2, -2])
theta_upper = np.array([2, 2])

### Posterior Specification

In [4]:
# Create an MDN Posterior Approximator Object (Uses Default Settings from the Paper)
posterior = alenn.mdn.MDNPosterior()

# Add the Model, Priors, and Empirical Data to the Newly-created Object
posterior.set_model(model)
posterior.set_prior(priors)
posterior.load_data(empirical)

--------------------------------------
Successfully created a new MDN object:
--------------------------------------
Number of lags:                3                   
Number of mixture components:  16                  
Number of neurons per layer:   32                  
Number of hidden layers:       3                   
Batch size:                    512                 
Number of epochs:              12                  
Activation function:           relu                
Input noise:                   0.2                 
Output noise:                  0.2                 
--------------------------------------

Model function successfully set.
----------------------------------------------------------------------------

Model prior successfully set. The model has 2 free parameters.
----------------------------------------------------------------------------

Empirical data successfully loaded. There are 999 observations in total.
--------------------------------------------------

### Sampler Specification

In [7]:
# Create an Adaptive MCMC Sampler Object
sampler = alenn.mcmc.AdaptiveMCMC(K = 70, S = 5000)

# Add the Posterior Approximator and Parameter Ranges to the Newly-created Object
sampler.set_posterior(posterior)
sampler.set_initialisation_ranges(theta_lower, theta_upper)

# Initiate the Sampling Process
sampler.sample_posterior()

-----------------------------------------------
Successfully created a new MCMC sampler object:
-----------------------------------------------
Number of sample sets:         5000                
Number of samples per set:     70                  
-----------------------------------------------

MDNPosterior object successfully loaded.
----------------------------------------------------------------------------

Initialisation ranges successfully set.

           Lower Bound  Upper Bound
Parameter                          
1                   -2            2
2                   -2            2
----------------------------------------------------------------------------



### Result Processing

In [6]:
# Process the Sampler Output
samples = sampler.process_samples(burn_in = 1500)

# Calculate the Posterior Mean
pos_mean = samples[:, :posterior.num_param].mean(axis = 0)

# Calculate the Posterior Standard Deviation
pos_std = samples[:, :posterior.num_param].std(axis = 0)

# Construct a Result Table
result_table = pd.DataFrame(np.array([pos_mean, pos_std]).transpose(), columns = ['Posterior Mean', 'Posterior Std. Dev.'])
result_table.index.name = 'Parameter'
result_table.index += 1

# Display the Result Table
print('Final Estimation Results:')
print('')
print(result_table)

Final Estimation Results:

           Posterior Mean  Posterior Std. Dev.
Parameter                                     
1                0.503976             0.040158
2                0.691284             0.111356


## 2.5. Free Parameter Set 5

### Model Specification

In [3]:
# Specify the Simulated Data Characteristics
T_emp = 1000    # Pseudo-empirical series length 
T_sim = 1000    # Length of each Monte Carlo replication
n = 100         # Number of Monte Carlo replications

# Specify the Pseudo-Empirical Data
empirical = np.diff(alenn.models.random_walk(700, 0.5, 0.4, 1, 2, T_emp, 1, 1), axis = 0)[:, 0]

# Define the Candidate Model Function
def model(theta):
    return np.diff(alenn.models.random_walk(700, theta[0], theta[1], 1, 2, T_sim, n, 7), axis = 0)

# Define Parameter Priors
priors = [stats.uniform(loc = -2, scale = 4).pdf,
          stats.uniform(loc = -2, scale = 4).pdf]

# Define the Parameter Bounds
theta_lower = np.array([-2, -2])
theta_upper = np.array([2, 2])

### Posterior Specification

In [4]:
# Create an MDN Posterior Approximator Object (Uses Default Settings from the Paper)
posterior = alenn.mdn.MDNPosterior()

# Add the Model, Priors, and Empirical Data to the Newly-created Object
posterior.set_model(model)
posterior.set_prior(priors)
posterior.load_data(empirical)

--------------------------------------
Successfully created a new MDN object:
--------------------------------------
Number of lags:                3                   
Number of mixture components:  16                  
Number of neurons per layer:   32                  
Number of hidden layers:       3                   
Batch size:                    512                 
Number of epochs:              12                  
Activation function:           relu                
Input noise:                   0.2                 
Output noise:                  0.2                 
--------------------------------------

Model function successfully set.
----------------------------------------------------------------------------

Model prior successfully set. The model has 2 free parameters.
----------------------------------------------------------------------------

Empirical data successfully loaded. There are 999 observations in total.
--------------------------------------------------

### Sampler Specification

In [7]:
# Create an Adaptive MCMC Sampler Object
sampler = alenn.mcmc.AdaptiveMCMC(K = 70, S = 5000)

# Add the Posterior Approximator and Parameter Ranges to the Newly-created Object
sampler.set_posterior(posterior)
sampler.set_initialisation_ranges(theta_lower, theta_upper)

# Initiate the Sampling Process
sampler.sample_posterior()

-----------------------------------------------
Successfully created a new MCMC sampler object:
-----------------------------------------------
Number of sample sets:         5000                
Number of samples per set:     70                  
-----------------------------------------------

MDNPosterior object successfully loaded.
----------------------------------------------------------------------------

Initialisation ranges successfully set.

           Lower Bound  Upper Bound
Parameter                          
1                   -2            2
2                   -2            2
----------------------------------------------------------------------------



### Result Processing

In [6]:
# Process the Sampler Output
samples = sampler.process_samples(burn_in = 1500)

# Calculate the Posterior Mean
pos_mean = samples[:, :posterior.num_param].mean(axis = 0)

# Calculate the Posterior Standard Deviation
pos_std = samples[:, :posterior.num_param].std(axis = 0)

# Construct a Result Table
result_table = pd.DataFrame(np.array([pos_mean, pos_std]).transpose(), columns = ['Posterior Mean', 'Posterior Std. Dev.'])
result_table.index.name = 'Parameter'
result_table.index += 1

# Display the Result Table
print('Final Estimation Results:')
print('')
print(result_table)

Final Estimation Results:

           Posterior Mean  Posterior Std. Dev.
Parameter                                     
1                0.570045             0.040679
2                0.468396             0.127495


## 2.6. Free Parameter Set 6

### Model Specification

In [3]:
# Specify the Simulated Data Characteristics
T_emp = 1000    # Pseudo-empirical series length 
T_sim = 1000    # Length of each Monte Carlo replication
n = 100         # Number of Monte Carlo replications

# Specify the Pseudo-Empirical Data
empirical = np.diff(alenn.models.random_walk(700, 0.7, 0.4, 1, 2, T_emp, 1, 1), axis = 0)[:, 0]

# Define the Candidate Model Function
def model(theta):
    return np.diff(alenn.models.random_walk(700, theta[0], theta[1], 1, 2, T_sim, n, 7), axis = 0)

# Define Parameter Priors
priors = [stats.uniform(loc = -2, scale = 4).pdf,
          stats.uniform(loc = -2, scale = 4).pdf]

# Define the Parameter Bounds
theta_lower = np.array([-2, -2])
theta_upper = np.array([2, 2])

### Posterior Specification

In [4]:
# Create an MDN Posterior Approximator Object (Uses Default Settings from the Paper)
posterior = alenn.mdn.MDNPosterior()

# Add the Model, Priors, and Empirical Data to the Newly-created Object
posterior.set_model(model)
posterior.set_prior(priors)
posterior.load_data(empirical)

--------------------------------------
Successfully created a new MDN object:
--------------------------------------
Number of lags:                3                   
Number of mixture components:  16                  
Number of neurons per layer:   32                  
Number of hidden layers:       3                   
Batch size:                    512                 
Number of epochs:              12                  
Activation function:           relu                
Input noise:                   0.2                 
Output noise:                  0.2                 
--------------------------------------

Model function successfully set.
----------------------------------------------------------------------------

Model prior successfully set. The model has 2 free parameters.
----------------------------------------------------------------------------

Empirical data successfully loaded. There are 999 observations in total.
--------------------------------------------------

### Sampler Specification

In [7]:
# Create an Adaptive MCMC Sampler Object
sampler = alenn.mcmc.AdaptiveMCMC(K = 70, S = 5000)

# Add the Posterior Approximator and Parameter Ranges to the Newly-created Object
sampler.set_posterior(posterior)
sampler.set_initialisation_ranges(theta_lower, theta_upper)

# Initiate the Sampling Process
sampler.sample_posterior()

-----------------------------------------------
Successfully created a new MCMC sampler object:
-----------------------------------------------
Number of sample sets:         5000                
Number of samples per set:     70                  
-----------------------------------------------

MDNPosterior object successfully loaded.
----------------------------------------------------------------------------

Initialisation ranges successfully set.

           Lower Bound  Upper Bound
Parameter                          
1                   -2            2
2                   -2            2
----------------------------------------------------------------------------



### Result Processing

In [6]:
# Process the Sampler Output
samples = sampler.process_samples(burn_in = 1500)

# Calculate the Posterior Mean
pos_mean = samples[:, :posterior.num_param].mean(axis = 0)

# Calculate the Posterior Standard Deviation
pos_std = samples[:, :posterior.num_param].std(axis = 0)

# Construct a Result Table
result_table = pd.DataFrame(np.array([pos_mean, pos_std]).transpose(), columns = ['Posterior Mean', 'Posterior Std. Dev.'])
result_table.index.name = 'Parameter'
result_table.index += 1

# Display the Result Table
print('Final Estimation Results:')
print('')
print(result_table)

Final Estimation Results:

           Posterior Mean  Posterior Std. Dev.
Parameter                                     
1                0.762023             0.039210
2                0.441793             0.152354
